<a href="https://colab.research.google.com/github/seamusrobertmurphy/01-data-processing/blob/main/02_global_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment setup

In [ ]:
!pip install leafmap geemap==0.30.4 geopandas numpy session_info rasterio
import ee, json, geemap, ipyleaflet, os, numpy, backports, session_info, json, tempfile, rasterio, time
import geopandas as gpd
import pandas as pd
from rasterio.mask import mask
from shapely.geometry import shape, mapping, Polygon
from google.colab import drive
from google.colab import files

!drive.mount('/content/drive')



### Activate Earth Engine

In [ ]:
#!ee.Authenticate() # deprecated in certain Colab environments
!earthengine authenticate
ee.Initialize(project = "murphys-deforisk")

### Jurisidictional boundaries

In [4]:
country = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state_list = country.aggregate_array('ADM1_NAME').distinct().getInfo()
state_list

['Grand Gedeh',
 'River Gee',
 'Gbarpolu',
 'Lofa',
 'Bomi',
 'Bong',
 'Grand Bassa',
 'Grand Cape Mount',
 'Grand Kru',
 'Margibi',
 'Maryland',
 'Montserrado',
 'Nimba',
 'Rivercess',
 'Sinoe']

In [14]:
red = {"color": "red", "width": 1.5, "lineType": "solid", "fillColor": "00000000"}
white = {"color": "white", "width": 0.5, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(
    country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(country, 7)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(country.style(**white), {}, "Country")
Map.addLayer(state.style(**white), {}, "States")
Map.add_labels(state,"ADM1_NAME",font_size="8pt",font_color="white")
Map.add_labels(country_label, "country_name", font_size="12pt", font_color="white", font_weight="bold",)
Map

Map(center=[6.446116651575274, -9.30729708708129], controls=(WidgetControl(options=['position', 'transparent_b…

In [29]:
aoi_gdf = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/AllLandscapes_merge_v02.shp')
aoi_gdf = aoi_gdf[aoi_gdf['Name'].isin(['Gola Forest National Park', 'Norman', 'Tonglay'])]
aoi_gdf = aoi_gdf[['Name', 'geometry']]
aoi_gdf['geometry'] = aoi_gdf['geometry'].simplify(tolerance=0.001)

def validate_and_repair_geometry(geom):
    if not geom.is_valid:
        geom = geom.buffer(0)
        if not geom.is_valid:
            print(f"Warning: Geometry is still invalid after repair: {geom}")
    return geom

projection = aoi_gdf.crs
print(f"Projection of the aoi_gdf object: {projection}")
aoi_gdf['area_ha'] = aoi_gdf['geometry'].area / 10000
aoi_gdf['geometry'] = aoi_gdf['geometry'].simplify(tolerance=0.01)

areas = aoi_gdf[['Name', 'area_ha']].copy()
total_area = areas['area_ha'].sum()
areas.loc['Total'] = pd.Series({'Name': 'Total', 'area_ha': total_area})
print(areas)


Projection of the aoi_gdf object: EPSG:32629
                            Name        area_ha
15     Gola Forest National Park   89050.392041
16                       Tonglay   19592.862480
17                        Norman   12297.851402
Total                      Total  120941.105923


In [22]:
with tempfile.TemporaryDirectory() as tempdir:
    temp_shp_path = os.path.join(tempdir, 'temp_aoi.shp')
    aoi_gdf.to_file(temp_shp_path)
    aoi = geemap.shp_to_ee(temp_shp_path)

Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(state.style(**white), {}, "States")
Map.addLayer(aoi.style(**red), {}, "AOI")
Map.add_labels(aoi,"Name",font_size="9pt",font_color="red",font_family="arial",font_weight="bold",)
Map


Map(center=[7.5441448221317655, -10.758250668984317], controls=(WidgetControl(options=['position', 'transparen…

The Dynamic World dataset provides near real-time (NRT) land cover classifications at a 10-meter resolution. It's important to note that this dataset's availability starts from June 27, 2015. Therefore, we can directly access data for 2019 and potentially 2024 (if data is available up to that year). However, for 2014, we'll need to use the closest available data.



In [43]:
# Load the Dynamic World dataset
dwCol = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
country_buff = country.geometry().buffer(10000)

# Dynamic World start date: 2015-07-27
dw2015 = dwCol.filterDate('2015-08-01', '2015-12-31').filterBounds(country_buff)
dw2019 = dwCol.filterDate('2019-08-01', '2019-12-31').filterBounds(country_buff)
dw2024 = dwCol.filterDate('2024-08-01', '2024-12-31').filterBounds(country_buff)
land_cover_2015 = dw2015.mode().select('label').clip(country_buff)
land_cover_2019 = dw2019.mode().select('label').clip(country_buff)
land_cover_2024 = dw2024.mode().select('label').clip(country_buff)

# Adjust land cover class names if needed
land_cover_classes = {
    0: 'Water',
    1: 'Trees',
    2: 'Grass',
    3: 'Flooded Vegetation',
    4: 'Crops',
    5: 'Shrub and Scrub',
    6: 'Built',
    7: 'Bare',
    8: 'Snow and Ice'
}

land_cover_colors = [
    '#419bdf', '#397d49', '#88b053', '#7a87c6',
    '#e49635', '#dfc35a', '#c4281b', '#a59b8f',
    '#b39fe1'
]

vis_params = {
    'min': 0,
    'max': 8,
    'palette': land_cover_colors
}

Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(land_cover_2015, vis_params, 'Land Cover 2015')
Map.addLayer(land_cover_2019, vis_params, 'Land Cover 2019')
Map.addLayer(land_cover_2024, vis_params, 'Land Cover 2024')
legend_dict = {land_cover_classes[i]: land_cover_colors[i] for i in range(len(land_cover_classes))}
Map.add_legend(legend_title="Land Cover Classes", legend_dict=legend_dict)
Map.addLayer(aoi.style(**white), {}, "AOI")
Map.add_labels(aoi,"Name",font_size="10pt",font_color="red",font_family="arial",font_weight="bold",)
Map

Map(center=[7.5441448221317655, -10.758250668984317], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
## CHUNK FOR EXPORTING Dynamic World land cover raster layers
def export_dynamic_world(image, year):
    export_params = {
        'image': image,
        'description': f'land_cover_{year}',
        'folder': '/content/drive/MyDrive/Colab Notebooks/outputs',
        'fileNamePrefix': f'land_cover_{year}',
        'scale': 10,
        'region': image.geometry().bounds(),
        'crs': image.projection().crs().getInfo(),
        'maxPixels': 1e13,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    while task.active():
        print(f'Exporting land cover for {year}... Status: {task.status()["state"]}')
        time.sleep(10)
    print(f'Export for {year} completed.')
    return f'/content/drive/MyDrive/{export_params["folder"]}/{export_params["fileNamePrefix"]}.tif'


land_cover_files = {}
land_cover_files[2015] = export_dynamic_world(land_cover_2015, 2015)
land_cover_files[2019] = export_dynamic_world(land_cover_2019, 2019)
land_cover_files[2024] = export_dynamic_world(land_cover_2024, 2024)

Exporting land cover for 2019... Status: RUNNING
Exporting land cover for 2019... Status: RUNNING
Exporting land cover for 2019... Status: RUNNING
Export for 2019 completed.
Exporting land cover for 2024... Status: READY
Exporting land cover for 2024... Status: RUNNING
Exporting land cover for 2024... Status: RUNNING


### Export to Drive

Caution, running the following chunk will export 12GB of duplicate rasters to the currently assigned drive folder.

In [ ]:
## CHUNK FOR EXPORTING processed Landsat bands
from datetime import datetime

# extract pathrow and date from scene ID
def get_pathrow_date(image_collection):
  first_image = image_collection.first()
  scene_id = first_image.get('system:index').getInfo()
  parts = scene_id.split('_')
  pathrow = parts[1]
  date_str = parts[2]
  date_obj = datetime.strptime(date_str, '%Y%m%d')
  date = date_obj.strftime('%Y-%m-%d')
  return pathrow, date

# define export parameters
def define_export_params(image, pathrow, date, band_name):
  description = f'composite_{date}_{band_name if isinstance(band_name, str) else "RGB"}_export'[:100]
  return {
    'image': image.select(band_name),
    'description': description,
    'folder': 'VT0007-deforestation-map',
    'fileNamePrefix': f'LANDSAT_TM-ETM-OLI_{pathrow}_{band_name if isinstance(band_name, str) else "RGB"}_{date}',
    'scale': 30,
    'region': state.geometry(),
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
  }

# get pathrow and date for each collection
pathrow_2014, date_2014 = get_pathrow_date(collection_2014)
pathrow_2019, date_2019 = get_pathrow_date(collection_2019)
pathrow_2024, date_2024 = get_pathrow_date(collection_2024)

# get band names
bandNames_2014 = composite_2014.bandNames().getInfo()
bandNames_2019 = composite_2019.bandNames().getInfo()
bandNames_2024 = composite_2024.bandNames().getInfo()

# export to cloud bucket
for band_name in bandNames_2014:
    export_params_2014 = define_export_params(composite_2014, pathrow_2014, date_2014, band_name)
    task_2014 = ee.batch.Export.image.toDrive(**export_params_2014)
    task_2014.start()
    print(f"Exporting 2014 image for band {band_name}. Task ID: {task_2014.id}")

for band_name in bandNames_2019:
    export_params_2019 = define_export_params(composite_2019, pathrow_2019, date_2019, band_name)
    task_2019 = ee.batch.Export.image.toDrive(**export_params_2019)
    task_2019.start()
    print(f"Exporting 2019 image for band {band_name}. Task ID: {task_2019.id}")

for band_name in bandNames_2024:
    export_params_2024 = define_export_params(composite_2024, pathrow_2024, date_2024, band_name)
    task_2024 = ee.batch.Export.image.toDrive(**export_params_2024)
    task_2024.start()
    print(f"Exporting 2024 image for band {band_name}. Task ID: {task_2024.id}")

export_params_2014_rgb = define_export_params(composite_2014, pathrow_2014, date_2014, ['RED', 'GREEN', 'BLUE'])
export_params_2014_rgb['image'] = composite_2014.visualize(**rgbVis)
task_2014_rgb = ee.batch.Export.image.toDrive(**export_params_2014_rgb)
task_2014_rgb.start()
print(f"Exporting 2014 RGB image. Task ID: {task_2014_rgb.id}")

export_params_2019_rgb = define_export_params(composite_2019, pathrow_2019, date_2019, ['RED', 'GREEN', 'BLUE'])
export_params_2019_rgb['image'] = composite_2019.visualize(**rgbVis)
task_2019_rgb = ee.batch.Export.image.toDrive(**export_params_2019_rgb)
task_2019_rgb.start()
print(f"Exporting 2019 RGB image. Task ID: {task_2019_rgb.id}")

export_params_2024_rgb['image'] = composite_2024.visualize(**rgbVis)
task_2024_rgb = ee.batch.Export.image.toDrive(**export_params_2024_rgb)
task_2024_rgb.start()
print(f"Exporting 2024 RGB image. Task ID: {task_2024_rgb.id}")

Exporting 2014 RGB image. Task ID: IIMNP6IZ4ALUYMXEILXGCRND


In [ ]:
session_info.show()

In [42]:
# Function to prepare/align objects and calculate class areas through:
# 1. Export the image to a temporary GeoTIFF file
# 2. Read the GeoTIFF as a GeoDataFrame using rasterio and geopandas
# 3. Reproject to aoi_gdf CRS (if necessary)
# 4. Calculate area by class

def calculate_area_by_class(image, aoi_gdf, year):
    with tempfile.TemporaryDirectory() as tempdir:
        temp_tif_path = os.path.join(tempdir, 'temp_image.tif')
        geemap.ee_export_image(
            image,
            filename=temp_tif_path,
            region=aoi_gdf.total_bounds,
            crs=aoi_gdf.crs,
            scale=10,
        )
        with rasterio.open(temp_tif_path) as src:
            data = src.read(1)
            transform = src.transform
            crs = src.crs
            shapes = []
            for i in range(data.shape[0]):
                for j in range(data.shape[1]):
                    if data[i, j] != src.nodata:
                        x, y = transform * (j, i)
                        polygon = Polygon([(x, y), (x + transform[0], y),
                                           (x + transform[0], y + transform[1]),
                                           (x, y + transform[1])])
                        shapes.append((data[i, j], polygon))
            landcover_gdf = gpd.GeoDataFrame(shapes, columns=['label', 'geometry'], crs=crs)
    landcover_gdf = landcover_gdf.to_crs(aoi_gdf.crs)
    landcover_with_subregions = gpd.sjoin(landcover_gdf, aoi_gdf, how="inner", predicate="intersects")

    # Calculate area by class and subregion
    area_by_class_subregion = landcover_with_subregions.groupby(['Name', 'label'])['geometry'].apply(
        lambda geom: geom.area.sum() / 10000
    ).reset_index()

    # Pivot the table to the desired format
    area_by_class_subregion = area_by_class_subregion.pivot(index='label', columns='Name', values='geometry')

    area_by_class_subregion.index = area_by_class_subregion.index.map(land_cover_classes)
    area_by_class_subregion.columns = [f'{col}_{year}' for col in area_by_class_subregion.columns]

    return area_by_class_subregion
